In [3]:
import pm4py
import os

log = pm4py.read_xes("../logs/sepsis.xes")

parsing log, completed traces ::   0%|          | 0/1050 [00:00<?, ?it/s]

In [6]:
from convert_log import convert_xes_n3

# convert_xes_n3(log, "../logs/sepsis_25.n3", 250)
# convert_xes_n3(log, "../logs/sepsis_50.n3", 500)
convert_xes_n3(log, "../logs/sepsis_all.n3")

total number of traces: 1050
conversion time (ms): 2327.367
save time (ms): 2070.646
total time (ms): 4398.191
